In [ ]:
import pandas as pd

In [ ]:
summary = pd.read_csv("data/clinvar/variant_summary.txt",delimiter="\t")

In [ ]:
goodStatus = ["criteria provided, single submitter",
              "criteria provided, multiple submitters, no conflicts",
              "reviewed by expert panel",
              "practice guideline",
              ]

In [ ]:
missense = summary[(summary.ReviewStatus.isin(goodStatus)) & (summary.Type == "single nucleotide variant")]

In [ ]:
missense

In [ ]:
from processBioDBs.utilities import getSequence

In [ ]:
from easydict import EasyDict

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
sequences = EasyDict()

In [ ]:
for symbol in tqdm(missense.GeneSymbol.unique()):
    if symbol not in sequences:
        try:
            sequences[symbol] = getSequence(symbol)
        except:
            continue